In [9]:




import findspark
findspark.init()
import pyspark # Call this only after findspark
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

#from pyspark.sql import SparkSession
import numpy as np
import pandas as pd


sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

#test sparkcontext
print(spark)

#https://spark.apache.org/docs/latest/sql-pyspark-pandas-with-arrow.html
#entry point for spark to interact with spark's functions
sparkSession = SparkSession.builder.appName("test_read_from_pg").getOrCreate()

#path to tweets text file in hadoop hdfs
data="hdfs://namenode:9000/user/test/input/tweets_pg_export.txt"

#https://stackoverflow.com/questions/49471192/spark-2-3-0-read-text-file-with-header-option-not-working
#load text file into Spark dataframes
df_load = sparkSession.read.option("header", "true").csv(data)

#test and print 2 rows of Spark dataframe
df_load.show(2, False)

# Enable Arrow-based columnar data transfers
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

#conver Spark dataframe into Pandas dataframe
df_pandas = df_load.select("*").toPandas()

print(df_pandas)

+------------------------------------------------+
|tweet_text                                      |
+------------------------------------------------+
|"@flawarah ""How Haze Affects Milk Tea Brand."""|
|The haze are really bad                         |
+------------------------------------------------+
only showing top 2 rows

                                             tweet_text
0      "@flawarah ""How Haze Affects Milk Tea Brand."""
1                               The haze are really bad
2     RT @allyxjackson: do yall understand just how ...
3     RT @BRITgrlINDOfood: Standing on a bridge over...
4        The haze is worse than when I arrived yester…"
...                                                 ...
4941                          @knowyouskz Sometime okay
4942                   @techinsider POLLUTION REDUCTION
4943  this is still the funniest shit ever esgdjgash...
4944  The weather and the haze also makes me feel th...
4945                  @techinsider  POLLUTION REDUCTI

In [10]:
#change tweets into lowercase
df_pandas['tweet_text'] = df_pandas['tweet_text'].str.lower()

print(df_pandas)

                                             tweet_text
0      "@flawarah ""how haze affects milk tea brand."""
1                               the haze are really bad
2     rt @allyxjackson: do yall understand just how ...
3     rt @britgrlindofood: standing on a bridge over...
4        the haze is worse than when i arrived yester…"
...                                                 ...
4941                          @knowyouskz sometime okay
4942                   @techinsider pollution reduction
4943  this is still the funniest shit ever esgdjgash...
4944  the weather and the haze also makes me feel th...
4945                  @techinsider  pollution reduction

[4946 rows x 1 columns]


In [12]:
#change tweets into lowercase
df_pandas['tweet_text'] = df_pandas['tweet_text'].str.lower()
#Removing RT retweet term
df_pandas['tweet_text'] = df_pandas['tweet_text'].str.replace('rt', '')
#Removing usernames
df_pandas['tweet_text'] = df_pandas['tweet_text'].replace(r'@\w+', '', regex=True)
#Removing url links
df_pandas['tweet_text'] = df_pandas['tweet_text'].replace(r'http\S+', '', regex=True)
df_pandas['tweet_text'] = df_pandas['tweet_text'].replace(r'www.[^ ]+', '', regex=True)
#remove next line \n
df_pandas['tweet_text'] = df_pandas['tweet_text'].replace('\n',' ', regex=True)
#remove numbers
df_pandas['tweet_text'] = df_pandas['tweet_text'].replace(r'[0-9]+', '', regex=True)
#removing special characters
df_pandas['tweet_text'] = df_pandas['tweet_text'].replace(r'[!"#$%&()*+,-./:;<=>?@[\]^_`{|}~]', '', regex=True)
# Removing tweet topic
df_pandas['tweet_text'] = df_pandas['tweet_text'].str.replace('haze', '')

print(df_pandas)

                                             tweet_text
0                           how  affects milk tea brand
1                                   the  are really bad
2       do yall understand just how bad the  is ther...
3       standing on a bridge overlooking the batangh...
4             the  is worse than when i arrived yester…
...                                                 ...
4941                                      sometime okay
4942                                pollution reduction
4943  this is still the funniest shit ever esgdjgash...
4944    the weather and the  also makes me feel thirsty
4945                                pollution reduction

[4946 rows x 1 columns]
